In [33]:
import json
import pandas as pd
import requests

In [34]:
# get players
players = requests.get('https://api.sleeper.app/v1/players/nfl').json()

with open('data/players.json', 'w') as f:
    json.dump(players, f, indent=4)

In [35]:
# get all players week stats
weekly = requests.get('https://api.sleeper.com/stats/nfl/2022/1?season_type=regular&position[]=DEF&position[]=K&position[]=QB&position[]=RB&position[]=TE&position[]=WR').json()

with open('data/weekly-2022-1.json', 'w') as f:
    json.dump(weekly, f, indent=4)

In [36]:
# process players file
players_df = pd.read_json('data/players.json').transpose()[['first_name', 'last_name', 'position', 'team', 'status', 'injury_status', 'active', 'player_id']]

# filter players
players_df = players_df[players_df['position'].isin(['QB', 'RB', 'WR', 'TE', 'K', 'DEF'])]
players_df = players_df[players_df['active']]

# project players
players_df['name'] = players_df['first_name'] + " " + players_df['last_name']
players_df = players_df[['name', 'position', 'team', 'status', 'injury_status', 'player_id']]
players_df.head()

,name,position,team,status,injury_status,player_id
5870,Daniel Jones,QB,NYG,Active,None,5870
6343,Manny Wilkins,QB,None,Active,None,6343
8595,Kyric McGowan,WR,WAS,Active,None,8595
3199,Michael Thomas,WR,NO,Active,None,3199
6487,Charles Scarff,TE,None,Inactive,None,6487


In [37]:
# process weekly file
with open('data/weekly-2022-1.json') as f:
    weekly = json.load(f)

weekly_df = pd.json_normalize(weekly).drop(columns=[
    'team', 'sport', 'season_type', 'company', 'category', 'game_id', 'date',
    'player.years_exp', 'player.news_updated', 'player.metadata', 'player.injury_status', 'player.injury_start_date', 'player.injury_notes', 'player.injury_body_part', 'player.fantasy_positions'
])

weekly_df.head()

,week,season,player_id,opponent,stats.tm_st_snp,stats.tm_off_snp,stats.tm_def_snp,stats.gms_active,player.team,player.position,...,stats.yds_allow_300_349,stats.pts_allow_14_20,stats.pts_allow_28_34,stats.yds_allow_500_549,stats.def_st_ff,stats.yds_allow_400_449,stats.def_td,stats.blk_kick,stats.def_st_fum_rec,stats.pts_allow_1_6
0,1,2022,1029,HOU,24.0,92.0,70.0,1.0,IND,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022,1034,LAC,21.0,58.0,67.0,1.0,LV,RB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2022,1049,NYG,23.0,65.0,60.0,1.0,TEN,QB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2022,1067,WAS,17.0,70.0,77.0,1.0,JAX,WR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2022,1099,NYG,23.0,65.0,60.0,1.0,TEN,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# what defenses scored the best, and against what team?
defenses = weekly_df[weekly_df['player.position'] == 'DEF']
defenses[['player_id', 'opponent', 'stats.pts_std']].sort_values(by=['stats.pts_std', 'player_id'], ascending=False)

,player_id,opponent,stats.pts_std
744,PIT,CIN,27.0
737,MIA,NE,20.0
721,BUF,LAR,17.0
734,LAC,LV,15.0
747,TB,DAL,13.0
738,MIN,GB,13.0
720,BAL,NYJ,13.0
748,TEN,NYG,11.0
735,LAR,BUF,11.0
723,CHI,SF,11.0
